In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp


In [ ]:
d=2

N=4
sigma = 0
k = 10


t_max = 100
time_steps = 1000 +1

t, dt = np.linspace(0,t_max, time_steps,retstep=True)


noise_scale =0.001*np.sqrt(dt)


def f(t, y):
    r = y[:d*N].reshape([-1,d])
    print(t)
    print(r)
    v = y[d*N:].reshape([-1,d])
    eta = np.random.normal(0,noise_scale,size=[N*d]).reshape([N,d])
    dv = -sigma*v + k*(np.roll(r, 1, axis=0) + np.roll(r, -1, axis=0) - 2*r) + eta
    
    dv[0] = -sigma*v[0] + k*(r[1] - r[0]) + eta[0]
    dv[-1] = -sigma*v[-1] + k*(r[-2] - r[-1]) + eta[-1]

    dr = v
    dy = np.stack((dr,dv)).flatten()
    print(dy)
    return dy



y0 = np.zeros([2*N*d])

x0 = np.linspace(0,1,N)
for i in range(0,N):
    y0[i*d] = x0[i]
print(y0)
# sol = solve_ivp(f, (0,t_max), y0, t_eval=t)

# print(sol)
# y = sol.y.T


#euler forward:
y = np.zeros([time_steps, 2*N*d])
y[0] = y0
for i in range(1,time_steps):
    y[i] = y[i-1] + dt * f(i, y[i-1])



print(y.shape)
r = y[:, :d*N].reshape([time_steps, -1,d])
v = y[:, d*N:].reshape([time_steps, -1,d])
print_steps = 10
for t in range(0,time_steps, time_steps//print_steps):
    plt.scatter(r[t,:,0], r[t,:,1])
    plt.show()



In [ ]:
from warnings import simplefilter


simplefilter(action="ignore", category=FutureWarning)
import diffrax
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt






d=2
N=40
sigma = 0
k = 1
noise_ampl=1


# t_max = 100
# time_steps = 10000 +1

# t, dt = jnp.linspace(0,t_max, time_steps,retstep=True)


# noise_scale =0.001*jnp.sqrt(dt)






t0, t1 = 0.0, 10
dt0 = 1e-4
saveat = diffrax.SaveAt(steps=True)

# Parameters
gamma = 10 #jnp.array([2, 0.5], dtype=jnp.float32)
u= noise_ampl / gamma
u = 1 #jnp.array([0.5, 2], dtype=jnp.float32)
x0 = jnp.zeros((N, d,), dtype=jnp.float32)
x0 = x0.at[:,0].set(
    jnp.linspace(0,1,N))
# print(x0)
v0 = jnp.zeros((N, d,), dtype=jnp.float32)
y0 = (x0, v0)

# Brownian motion
bm = diffrax.VirtualBrownianTree(
    t0, t1, tol=0.01, shape=(N, d,), key=jr.key(0), levy_area=diffrax.SpaceTimeTimeLevyArea
)

fconst = k/u
def f(x, args):
    
    r = jnp.reshape(x, [-1,d])
    
    dv =  (jnp.roll(r, 1, axis=0,) + jnp.roll(r, -1, axis=0) - 2*r)  
    # print(dv)
    dv = dv.at[0].set((r[1] - r[0])) 
    dv = dv.at[-1].set((r[-2] - r[-1]))
    return fconst *dv

drift_term = diffrax.UnderdampedLangevinDriftTerm(gamma, u, f)
diffusion_term = diffrax.UnderdampedLangevinDiffusionTerm(gamma, u, bm)
terms = diffrax.MultiTerm(drift_term, diffusion_term)

solver = diffrax.QUICSORT(0.1)
# solver = diffrax.ALIGN()
# solver = diffrax.SlowRK()
sol = diffrax.diffeqsolve(
    terms, solver, t0, t1, dt0=dt0, y0=y0, args=None, saveat=saveat, max_steps=1000000
)
xs, vs = sol.ys


print(xs.shape)


In [ ]:
print_steps = 10
for t in range(0,time_steps, time_steps//print_steps):
    plt.scatter(xs[t,:,0], xs[t,:,1])
    plt.show()

In [ ]:
# Plot the trajectory against time and velocity against time in a separate plot
dim=0
fig, axs = plt.subplots(2, 1, figsize=(10, 10))
# axs[0].plot(sol.ts, xs[:, 0, dim], label="x1")
# axs[0].plot(sol.ts, xs[:, 1, dim], label="x2")
# axs[0].plot(sol.ts, xs[:, 2, dim], label="x3")
# axs[0].plot(sol.ts, xs[:, 3, dim], label="x4")
axs[0].plot(sol.ts, jnp.mean(xs, axis=1)[:,dim],color='k', label="mean")
axs[0].plot(sol.ts, jnp.abs(xs[:, 0, dim] - xs[:, -1, dim]), label="|xN - x1|")
axs[0].plot(sol.ts, jnp.abs(xs[:, 0, dim] - xs[:, -1, dim]), label="|xN - x1|")
axs[0].set_xlabel("Time")
axs[0].set_ylabel("Position")
axs[0].legend()
axs[0].grid()

axs[1].plot(sol.ts, vs[:, 0, dim], label="x1")
axs[1].plot(sol.ts, vs[:, 1, dim], label="x2")
axs[1].plot(sol.ts, vs[:, 2, dim], label="x3")
axs[1].plot(sol.ts, vs[:, 3, dim], label="x4")
axs[1].set_xlabel("Time")
axs[1].set_ylabel("Velocity")
axs[1].legend()
axs[1].grid()